In [7]:
import os

import numpy as np

from sklearn.metrics import classification_report, confusion_matrix

from keras.applications import nasnet, resnet50, inception_v3
from keras.preprocessing import image
from keras import losses, metrics, callbacks, optimizers, activations, models, layers
from keras import backend as K
K.set_image_dim_ordering('tf')

def _nasnet(num_classes, pretrained=True, freezed=True):
    weights = 'imagenet' if pretrained else None
    base_model = nasnet.NASNetMobile(weights=weights, include_top=False)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    predictions = layers.Dense(num_classes, activation=activations.softmax, name='predictions')(x)
    if freezed:
        for layer in base_model.layers:
            layer.trainable = False
    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

def _inception(num_classes, pretrained=True, freezed=True):
    weights = 'imagenet' if pretrained else None
    base_model = inception_v3.InceptionV3(weights=weights, include_top=False)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation=activations.relu)(x)
    predictions = layers.Dense(num_classes, activation=activations.softmax, name='predictions')(x)
    if freezed:
        for layer in base_model.layers:
            layer.trainable = False
    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

def _resnet(num_classes, pretrained=True, freezed=True):
    weights = 'imagenet' if pretrained else None
    base_model = resnet50.ResNet50(input_shape=(224,224,3), weights=weights, include_top=False)
    x = base_model.output
    x = layers.Flatten()(x)
    predictions = layers.Dense(num_classes, activation=activations.softmax, name='predictions')(x)
    if freezed:
        for layer in base_model.layers:
            layer.trainable = False
    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model


dataset_path = '/home/daniel/Downloads/CBMS/exp5-1'

for i in range(1):
    img_size = 224
    batch_size = 32
    generator = image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
    #                                     ,shear_range=0.2,
    #                                     zoom_range=0.2,
    #                                     horizontal_flip=True)

    train_data_gen = generator.flow_from_directory(dataset_path, 
                                                   #color_mode='grayscale', 
                                                   target_size=(img_size,img_size),
                                                   subset='training', 
                                                   batch_size=batch_size)

    test_data_gen = generator.flow_from_directory(dataset_path, 
                                                  #color_mode='grayscale', 
                                                  target_size=(img_size,img_size), 
                                                  subset='validation', 
                                                  batch_size=1,
                                                  shuffle=False)

    model = None
    model_ckp = os.path.basename(dataset_path) + '_resnet_ckp_' + str(i) + '.hdf5'

    checkpointer = callbacks.ModelCheckpoint(filepath=model_ckp, verbose=1, save_best_only=True)
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    tensorboard = callbacks.TensorBoard()
    progbar = callbacks.ProgbarLogger()
    earlystop = callbacks.EarlyStopping(patience=8)
    csv_logger = callbacks.CSVLogger(os.path.basename(dataset_path) + '_resnet_' + str(i) + '.csv')

    # if os.path.exists(model_ckp):
    #     model = models.load_model(model_ckp)
    # else:
    model = _resnet(train_data_gen.num_classes, freezed=True)
    model.compile(optimizer=optimizers.Adam(lr=0.01), 
                  loss=losses.categorical_crossentropy, 
                  metrics=[metrics.categorical_accuracy])

#     model.summary()

    hist = model.fit_generator(train_data_gen, epochs=50, validation_data=test_data_gen, workers=8, verbose=0,
                               callbacks=[checkpointer, csv_logger])


    prob = model.predict_generator(test_data_gen, verbose=1)
    y_pred = np.argmax(prob, axis=1)
    y_true = test_data_gen.classes

    #     print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    np.savetxt(os.path.basename(dataset_path) + '_resnet_confusion_matrix_' + str(i) + '.csv',
               confusion_matrix(y_true, y_pred).astype(int), delimiter=',')

Found 2674 images belonging to 5 classes.
Found 665 images belonging to 5 classes.

Epoch 00001: val_loss improved from inf to 3.16245, saving model to exp5-1_resnet_ckp_0.hdf5

Epoch 00002: val_loss improved from 3.16245 to 1.20179, saving model to exp5-1_resnet_ckp_0.hdf5

Epoch 00003: val_loss improved from 1.20179 to 1.15036, saving model to exp5-1_resnet_ckp_0.hdf5

Epoch 00004: val_loss did not improve

Epoch 00005: val_loss did not improve

Epoch 00006: val_loss did not improve

Epoch 00007: val_loss did not improve

Epoch 00008: val_loss did not improve

Epoch 00009: val_loss did not improve

Epoch 00010: val_loss did not improve

Epoch 00011: val_loss did not improve

Epoch 00012: val_loss did not improve

Epoch 00013: val_loss did not improve

Epoch 00014: val_loss did not improve

Epoch 00015: val_loss did not improve

Epoch 00016: val_loss did not improve

Epoch 00017: val_loss did not improve

Epoch 00018: val_loss did not improve

Epoch 00019: val_loss did not improve

Ep

In [8]:

model_features = models.Model(inputs=model.input, outputs=model.get_layer('#layer-name#').output)

features = model_features.predict_generator(test_data_gen)


ValueError: No such layer: #layer-name#

In [9]:
import os

path = '/home/daniel/Downloads/CBMS/exp5-1'

for i in os.listdir(path):
    print()
    
print(os.listdir('/home/daniel/Downloads/CBMS/exp5-1'))

['birads5', 'birads3', 'birads4', 'birads2', 'birads1']
